In [2]:
import torch
from homework4 import CNP
import numpy as np

In [ ]:
def generate_samples(traj, d_x):   
    random_rows = traj[np.random.choice(traj.shape[0], size=4, replace=False)]
    
    random_rows = np.array(random_rows)
    
    tensor_1 = torch.tensor(random_rows[:3], dtype=torch.double) 
    tensor_2 = torch.tensor([random_rows[3][:d_x]], dtype=torch.double) 
    tensor_3 = torch.tensor([[random_rows[3][d_x:]]], dtype=torch.double) 

    return tensor_1, tensor_2, tensor_3

def train():
    d_x = 3
    d_y = 2
    model = CNP([d_x,d_y], 128, 3)
    training_data = np.load("states.npy")
    traj_count = len(training_data)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    for i in range(100_000):
        traj = training_data[np.random.randint(traj_count)]
        traj = torch.tensor(traj, dtype=torch.float32)
        samples, target_x, target_y = generate_samples(traj)
        optimizer.zero_grad()
        output = model(samples[:3], target_x)
        loss = CNP.nll_loss(output, target_y)
        optimizer.zero_grad()
        loss = model.loss(traj)
        loss.backward()
        optimizer.step()
        



TypeError: __init__() missing 3 required positional arguments: 'in_shape', 'hidden_size', and 'num_hidden_layers'